In [1]:
from random import random
import numpy as np
import cv2
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
import skimage
import scipy
%matplotlib inline

In [58]:
from PIL import Image

In [2]:
def read_image(IMG_NAME):
    img = cv2.imread(IMG_NAME)
    return cv2.resize(img, (480, 360))

In [22]:
class Vertex:
    def __init__(self, parent, rank=0, size=1):
        self.p = parent
        self.r = rank
        self.s = size

class Universe:
    def __init__(self, num_v):
        self.v = [Vertex(i) for i in range(num_v)]
        self.num_v = num_v

    def size(self, i):
        return self.v[i].s

    def find(self, n):
        temp = n
        while temp != self.v[temp].p:
            temp = self.v[temp].p
        self.v[n].p = temp
        return temp

    def union(self, a, b):
        if self.v[a].r > self.v[b].r:
            self.v[b].p = a
            self.v[a].s += self.v[b].s
        else:
            self.v[a].p = b
            self.v[b].s += self.v[a].s
            if self.v[a].r == self.v[b].r:
                self.v[b].r += 1
        self.num_v -= 1

In [106]:
def threshold_func(size, k):
    return (k * 1.0 / size)

In [117]:
def build(img):
    h, w, _ = img.shape
    edges = []
    
    def create_edge(img, x, y, x1, y1):
        def vertex(x, y):
            return x * w + y
        return (vertex(x, y), vertex(x1, y1), np.sqrt(np.sum((img[x, y] - img[x1, y1]) ** 2)))

    for x in range(h):
        for y in range(w):
            if x > 0:
                edges.append(create_edge(img, x, y, x-1, y))
            if y > 0:
                edges.append(create_edge(img, x, y, x, y-1))
            if x > 0 and y > 0:
                edges.append(create_edge(img, x, y, x-1, y-1))
            if x > 0 and y < h-1:
                edges.append(create_edge(img, x, y, x-1, y+1))

    return edges

In [131]:
def generate_image(u, w, h):
    random_color = lambda: (int(random()*255), int(random()*255), int(random()*255))
    colors = [random_color() for i in range(w*h)]

    img = Image.new('RGB', (h, w))
    im = img.load()
    for x in range(h):
        for y in range(w):
            comp = u.find(x * w + y)
            im[x, y] = colors[comp]

    return img.transpose(Image.ROTATE_270).transpose(Image.FLIP_LEFT_RIGHT)

In [135]:
sigma = 1
k = 8000
min_size = 5000

In [139]:
def img_seg(IMG_NAME):
    img = read_image(IMG_NAME)
    img = scipy.ndimage.gaussian_filter(img, sigma = sigma)
    edges = build(img)

    weight = lambda edge: edge[2]
    sorted_edges = sorted(edges, key=weight)
    h, w, _ = img.shape
    num_v = h*w
    u = Universe(num_v)
    threshold = [threshold_func(1, k) for _ in range(num_v)]

    for edge in sorted_edges:
            p_a = u.find(edge[0])
            p_b = u.find(edge[1])
            a_condition = weight(edge) <= threshold[p_a]
            b_condition = weight(edge) <= threshold[p_b]

            if p_a != p_b and a_condition and b_condition:
                u.union(p_a, p_b)
                a = u.find(p_a)
                threshold[a] = weight(edge) + threshold_func(u.v[a].s, k)

    for edge in sorted_edges:
        a = u.find(edge[0])
        b = u.find(edge[1])
        if a != b and (u.size(a) < min_size or u.size(b) < min_size):
            u.union(a, b)

    output = generate_image(u, w, h)
    output.save("sef2.jpg")